In [ ]:
import pandas as pd
import numpy as np
import os
import os.path as osp
import subprocess
from src.data_funcs import get_stids
import json
from datetime import datetime, timedelta
from synoptic.services import stations_timeseries

In [ ]:
# Load Setup
tokens = json.load(open('etc/tokens.json'))
meso_token = tokens['mesowest']

# User Arugments
start = "202306010000" 
end = "202308312300"
bbox = [37,-111,46,-95]

In [ ]:
# Get STIDs 
sts=get_stids(start, end, bbox, meso_token)

In [ ]:
raws_vars = ["air_temp", "relative_humidity", "precip_accum", "fuel_moisture", "wind_speed", "solar_radiation"]
t0 = datetime.strptime(start, "%Y%m%d%H%M")
t1 = datetime.strptime(end, "%Y%m%d%H%M")
params = dict(
    stid=["PLACEHOLDER"], # change this in the loop
    vars=raws_vars,
    start=t0,
    end= t1+timedelta(hours=1) # add an hour since it doesn't include end date exactly
)

In [ ]:
# Loop over STIDs and collect data with full sensor data
for st in sts:

    print("~"*50)
    print(f"Collecting RAWS data for {st}")
    params["stid"] = [st]
    try:
        df_temp = stations_timeseries(verbose="HIDE", **params)

        if "fuel_moisture" in dat.columns:
            print("Collected FMC data")
            # loc, raws = format_raws_df(dat,tstart, tend)
            # title = f"{st}_{t0.year}{t0.strftime('%m')}"
            # out_dict[title] = {"loc":loc, "RAWS": raws}
        else:
            print(f"Not full data at {st}")
    except AssertionError as e:
        # Error handling behavior
        print("AssertionError caught:", e)